In [1]:
from google.colab import files, drive
import gdown

import json
import numpy

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -r sample_data

In [3]:
# Аудиозапись, которую нужно Транскрибировать и определить ее Класс
audio = '/content/drive/MyDrive/Media_108/Запись 3098496016.mp3'

# Импорт и инициализация для Whisper-транскрибации

In [ ]:
# Установка cohere openai tiktoken
!pip -q install cohere openai tiktoken
# Установка openai-whisper
!pip install git+https://github.com/openai/whisper.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 833.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xwjg460q
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xwjg460q
  Resolved https://github.com/openai/whisper.git to commit e58f28804528831904c3b6f2c0e473f346223433
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  C

In [ ]:
import whisper

In [ ]:
# Загрузка модели whisper medium
whisper_model = whisper.load_model('medium')

100%|██████████████████████████████████████| 1.42G/1.42G [00:10<00:00, 148MiB/s]


In [ ]:
# Возможное решение этого предупреждения
# /usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
#  warnings.warn("FP16 is not supported on CPU; using FP32 instead")

import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = whisper.load_model("base", device = "cpu" )

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 124MiB/s]


# Whisper-транскрибация

In [ ]:
text_whisper = whisper_model.transcribe(audio, language='ru')['text']

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
print(text_whisper)

 Здравствуйте, вы позвонили в группу компании Cortros. Благодарим вас за звонок. Пожалуйста, дождитесь ответа оператора. Группа компании Cortros. Меня зовут Елена. Здравствуйте. Здравствуйте. Скажите, пожалуйста, дома, которые у вас в готовом виде сейчас есть, в каком районе расположен? Ну или адрес назовите, я сам оторентирую. Смотрите, у нас три объекта. Первый – это жилой комплекс Headliner. Он расположен в Пресненском районе. Шмитовский проект 39, прямо рядом с метро Шелепиха. Второй жилой комплекс – это I-Lav, это улица Бочкова 11, Останкинский район, метро Алексеевская. И третий – это 11-стовский район, жилой комплекс Равновесия, под Москве. Ага, понятно. А вот тогда про первое скажите, вот там в квартире двухкомплексное, какого метража и цена? Есть ли квартира с отделкой и цена? По данному вопросу вас ориентирует менеджер отдела продаж, поскольку у нас сейчас проходят еще акции с киски, это лучше с ними обсудить. Я вас сейчас соединю и вам предоставлю информацию. Как вас зовут? 

# Импорт и инициализация для Bert-предсказания

In [5]:
import os            # для работы с операционной системой
import pandas as pd  # для работы с таблицами
import numpy  as np  # для работы с массивами
import torch

In [6]:
# Установка transformers
!pip -q install transformers

In [7]:
# Импортируем предобученную модель
from transformers import AutoModel
bert = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [8]:
# Сам BERT обучать не будем, добавим к его выходу свои слои, которые и будем обучать
for param in bert.parameters():
  param.requires_grad = False

import torch.nn as nn
class BERT_Arch(nn.Module):
  def __init__(self, bert):
    super(BERT_Arch, self).__init__()
    self.bert = bert
    self.dropout = nn.Dropout(0.1)
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(768,512)
    self.fc2 = nn.Linear(512,2)
    self.softmax = nn.LogSoftmax(dim = 1)
  def forward(self, sent_id, mask):
    _, cls_hs = self.bert(sent_id, attention_mask = mask, return_dict = False)
    x = self.fc1(cls_hs)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = self.softmax(x)
    return x

In [9]:
# Архитектура
bert.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [10]:
# Объявляем модель и загружаем её в GPU
device = torch.device('cuda')
model  = BERT_Arch(bert)
model  = model.to(device)

In [11]:
# Загрузим лучшие веса для модели
model.load_state_dict(torch.load('/content/drive/MyDrive/Media_108/Bert_веса/bert_weights.pt'))

<All keys matched successfully>

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

# Bert-предсказание

In [15]:
# Токенизируем текст
tokens = tokenizer.tokenize(text_whisper)

# Преобразуем токены в числовые идентификаторы
ids = tokenizer.convert_tokens_to_ids(tokens)

# Добавляем паддинг до максимальной длины последовательности
max_seq_len = 512
ids = ids[:max_seq_len - 2]  # учитываем [CLS] и [SEP]
ids = [tokenizer.cls_token_id] + ids + [tokenizer.sep_token_id]
mask = [1] * len(ids)
padding = [tokenizer.pad_token_id] * (max_seq_len - len(ids))
ids += padding
mask += padding

# Преобразуем числовые идентификаторы в тензор PyTorch
test_seq = torch.tensor(ids).unsqueeze(0)
test_mask = torch.tensor(mask).unsqueeze(0)

# Передаем тензор через модель, чтобы получить предсказание
with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))

# Преобразуем выход модели в вероятности классов
probs = torch.softmax(preds, dim=1)

In [18]:
# Извлекаем вероятность класса text_whisper
text_whisper_prob = probs[0, 1].item()

# Сохраняем вероятность в переменную confidence
confidence = text_whisper_prob

# Тк здесь вероятностная принадлежность классу (у нас их 2), то есть вероятность (0, 0.5) - это НЕЦЕЛЕВОЙ, а [0.5, 1] - ЦЕЛЕВОЙ

if confidence >= 0.5:
    print(f'Звонок, путь до которого {audio}, является ЦЕЛЕВЫМ, вероятность - {confidence}')
else:
    print(f'Звонок, путь до которого {audio}, является НЕЦЕЛЕВЫМ, вероятность - {1 - confidence}')

Звонок, путь до которого /content/drive/MyDrive/Media_108/Запись 3098496016.mp3, является ЦЕЛЕВЫМ, вероятность - 0.898822009563446
